In [1]:
import awswrangler as awr
import pandas as pd

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"
with open (file_path, 'r') as r:
    query = r.read()

df_sets = awr.athena.read_sql_query(query, database='silver')

CONFERÊNCIAS

In [2]:
df_sets['status_conjunto'].unique()
# se ATIVO NOVO RENOVAÇÃO é placa vigente, porque tem placas que nunca mudam para ATIVO, ficam como estão
# ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

<StringArray>
[            'FINALIZADO',                  'ATIVO',              'RENOVAÇÃO',
              'CANCELADO',            'DESISTÊNCIA',                   'NOVO',
 'DEVOLVIDO PARA UNIDADE',            'SUBSTITUÍDO', 'AGENDADO PARA ATIVAÇÃO',
  'AGUARDANDO ASSINATURA',      'AGUARDANDO ATIVAR',           'NAO RENOVADO',
                 'NEGADO',                     <NA>]
Length: 14, dtype: string

In [3]:
df_sets1 = df_sets[df_sets['placa']=='ZZZ9999']
df_sets1.sort_values('data_registro', ascending=False)

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
46142,ZZZ9999,528LKACKDXPPD244,186298,2149,2833,2024-09-03,None,2024-09-03,SUBSTITUÍDO,Viavante,2025-04-22


criando cancelamentos e ativações

In [4]:
df_ativacoes = df_sets[df_sets['data_cancelamento_conjunto'].isnull()]

df_ativacoes = df_ativacoes[df_ativacoes['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes['data_status'] = df_ativacoes['data_ativacao_conjunto']

df_ativacoes['vigencia'] = 'ATIVO'

In [5]:
df_cancelamentos = df_sets[df_sets['data_cancelamento_conjunto'].notna()]

df_cancelamentos['data_status'] = df_cancelamentos['data_cancelamento_conjunto']

df_cancelamentos['vigencia'] = 'CANCELADO'

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_21508\1030088362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos['data_status'] = df_cancelamentos['data_cancelamento_conjunto']
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_21508\1030088362.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos['vigencia'] = 'CANCELADO'


juntando as duas e filtrando ontem

In [9]:
df_aggregate = pd.concat([df_ativacoes, df_cancelamentos])


today = pd.Timestamp.today()
today_date = today.date()
yesterday = today - pd.Timedelta(days=1)
yesterday_date= yesterday.date()

df_aggregate['data_status'] = pd.to_datetime(df_aggregate['data_status'])
df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==yesterday_date]

#df_aggregate.sort_values('data_ativacao_conjunto', ascending=False)
df_aggregate_yesterday

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia
2444,ASI2378,9534N8244AR020875,24986,17657,14141,2025-04-14,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
4175,ARZ7274,9BVAS02C3AE753672,25053,15293,14176,2025-04-16,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
7994,RBL1D24,9BSR6X400L3974985,24928,19994,14203,2025-04-22,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
9898,RYH3H50,9EP070820P1001426,16791,10852,14413,2025-05-12,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
9899,RYH3H60,9EP070820P1001427,16790,10852,14413,2025-05-12,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58085,RDV6F85,93ZS3HUH0L8834509,207246,3146,4298,2024-10-18,2025-05-14,2025-05-14,CANCELADO,Viavante,2025-05-14,2025-05-14,CANCELADO
58232,RHW8J80,953658246NR046019,207486,3275,4483,2024-10-24,2025-05-14,2024-10-22,CANCELADO,Viavante,2025-05-14,2025-05-14,CANCELADO
58252,JTA9G08,9BSTH4X2ZP3249631,207646,3290,4504,2024-10-24,2025-05-14,2024-10-24,CANCELADO,Viavante,2025-05-14,2025-05-14,CANCELADO
58254,ATF0J48,93ZS2MSH0B8809993,207630,3292,4508,2024-10-24,2025-05-14,2024-10-24,CANCELADO,Viavante,2025-05-14,2025-05-14,CANCELADO


gerando a base geral de ativos

In [ ]:
import pandas as pd
import awswrangler as awr
import openpyxl

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets_active.sql"

with open (file_path, 'r') as file:
    query = file.read()




df_all_active = awr.athena.read_sql_query(query, database='silver')



,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
0,MKM1I71,9535N8246CR200625REM,13518,17258,12622,2024-12-04,None,2024-12-11,ATIVO,Stcoop,2024-12-18
1,MFQ0C54,9EP07102081002960,7679,17283,12611,2024-12-04,None,2024-12-06,ATIVO,Stcoop,2024-12-09
2,MMC9D64,9AA07133GEC126820,2571,17284,13730,2025-02-24,None,2025-02-26,ATIVO,Stcoop,2025-04-17
3,MLD9B48,9EP071120D1001811,9357,17284,12690,2024-12-10,None,2024-12-27,ATIVO,Stcoop,2025-02-07
4,QZQ8B89,9A9RSM3TBPLDV2191,18234,17313,10451,2024-08-14,None,2024-08-16,ATIVO,Stcoop,2024-09-26
...,...,...,...,...,...,...,...,...,...,...,...
31773,SDL8I21,9ADG0942RRM538647,9031,8280,13948,2025-04-10,None,2025-04-10,ATIVO,Viavante,2025-04-10
31774,SDM8E71,9ADG0942RRM538646,9032,8280,13948,2025-04-10,None,2025-04-10,ATIVO,Viavante,2025-04-10
31775,QWD7H55,9ADG0942LLM455816,9023,8280,13948,2025-04-10,None,2025-04-10,ATIVO,Viavante,2025-04-10
31776,RLK2H02,9536K824XNR000153,15985944,8282,13950,2025-04-10,None,2025-04-10,ATIVO,Viavante,2025-04-22


concatenando base geral (menos a de ontem e hoje) movimentações ontem

In [15]:
df_all_active['data_ativacao_conjunto'] = pd.to_datetime(df_all_active['data_ativacao_conjunto'])
df_all_active['data_cancelamento_conjunto'] = pd.to_datetime(df_all_active['data_cancelamento_conjunto'])

df_all_active = df_all_active[
    (df_all_active['data_ativacao_conjunto'].dt.date!=yesterday_date) &
    (df_all_active['data_ativacao_conjunto'].dt.date!=today_date) &
    (df_all_active['data_cancelamento_conjunto'].dt.date!=yesterday_date) &
    (df_all_active['data_cancelamento_conjunto'].dt.date!=today_date)
    ]

df_all = pd.concat([df_aggregate_yesterday, df_all_active])
df_all

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia
2444,ASI2378,9534N8244AR020875,24986,17657,14141,2025-04-14,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
4175,ARZ7274,9BVAS02C3AE753672,25053,15293,14176,2025-04-16,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
7994,RBL1D24,9BSR6X400L3974985,24928,19994,14203,2025-04-22,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
9898,RYH3H50,9EP070820P1001426,16791,10852,14413,2025-05-12,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
9899,RYH3H60,9EP070820P1001427,16790,10852,14413,2025-05-12,None,2025-05-14,ATIVO,Stcoop,2025-05-15,2025-05-14,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31773,SDL8I21,9ADG0942RRM538647,9031,8280,13948,2025-04-10,NaT,2025-04-10 00:00:00,ATIVO,Viavante,2025-04-10,NaT,NaN
31774,SDM8E71,9ADG0942RRM538646,9032,8280,13948,2025-04-10,NaT,2025-04-10 00:00:00,ATIVO,Viavante,2025-04-10,NaT,NaN
31775,QWD7H55,9ADG0942LLM455816,9023,8280,13948,2025-04-10,NaT,2025-04-10 00:00:00,ATIVO,Viavante,2025-04-10,NaT,NaN
31776,RLK2H02,9536K824XNR000153,15985944,8282,13950,2025-04-10,NaT,2025-04-10 00:00:00,ATIVO,Viavante,2025-04-22,NaT,NaN


criando as condições do status baeado no histórico montado

In [ ]:
df_all['status'] = pd.NA

if df_all['data_ativacao_conjunto'].dt.date==yesterday_date:
    if df_all['vigencia'] == 'ATIVO'

In [ ]:

status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO'] 


# Inicializar colunas de status e migration_from
df['status_beneficio'] = 'NOVO'
df['migration_from'] = np.nan

# Agrupar dados de conferência por chassi e benefício
df_conf_grouped = df_conf.groupby(['chassi', 'beneficio']).agg({
    'empresa': list,
    'data_ativacao_beneficio': list,
    'status_beneficio': list
}).reset_index()

# Criar um dicionário para lookup rápido
conf_dict = df_conf_grouped.set_index(['chassi', 'beneficio']).to_dict('index')

# Processar cada chassi/benefício único (validação)
for chassi, beneficio in df[['chassi', 'beneficio']].drop_duplicates().values:
    if (chassi, beneficio) in conf_dict:
        conf_data = conf_dict[(chassi, beneficio)]
        
        if len(conf_data['empresa']) > 1:
            # Ordenar datas e status
            dates = sorted([d for d in conf_data['data_ativacao_beneficio'] if pd.notna(d)])
            
            if len(dates) > 1:
                penultima_data = dates[-2]
                idx_penultima = conf_data['data_ativacao_beneficio'].index(penultima_data)
                status_penultimo = conf_data['status_beneficio'][idx_penultima]
                empresa_penultima = conf_data['empresa'][idx_penultima]
                
                # Atualizar registros correspondentes
                mask = (df['chassi'] == chassi) & (df['beneficio'] == beneficio)
                
                if status_penultimo not in status_filter_list:
                    if empresa_penultima != df.loc[mask, 'empresa'].iloc[0]:
                        df.loc[mask, 'status_beneficio'] = 'MIGRAÇÃO'
                        df.loc[mask, 'migration_from'] = empresa_penultima
                    else:
                        df.loc[mask, 'status_beneficio'] = 'RENOVAÇÃO'
                else:
                    df.loc[mask, 'status_beneficio'] = 'REATIVAÇÃO'

